# MPC Simulation and Analysis

This notebook provides an interactive interface for running and analyzing MPC simulations. You can:
1. Configure simulation parameters
2. Run the simulation
3. Visualize results with interactive plots
4. Save results for later analysis

First, let's import the necessary modules and set up our environment.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
from datetime import datetime
from pathlib import Path

# Import our simulation module
from run_numeric_sim import NumericSimulator, create_default_config

# Configure matplotlib for notebook
%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100


/tmp/ipykernel_768982/1137829578.py:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


## Configuration

Let's set up the simulation configuration. We'll create a default configuration file and then customize it for our needs.


In [2]:
# Create a default configuration
config_file = 'numeric_sim_config.yaml'
create_default_config(config_file)

# Load and display the configuration
with open(config_file, 'r') as f:
    config = yaml.safe_load(f)
print("Default configuration:")
print(yaml.dump(config, default_flow_style=False))


Default configuration created: numeric_sim_config.yaml
Default configuration:
disturbance_force:
- 0.0
- 0.0
- 0.0
disturbance_start_time: 5.0
disturbance_torque:
- 0.0
- 0.0
- 0.0
dt_traj_opt: 50
enable_disturbance: false
enable_noise: false
initial_state:
- 0.0
- 0.0
- 1.0
- 0.0
- 0.0
- 0.0
- 1.0
- -1.2
- -0.6
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
plot_results: true
results_dir: results/numeric_sim
robot_name: s500_uam
save_results: true
simulation_time: 10.0
state_noise_std: 0.001
trajectory_name: hover
use_squash: true
yaml_path: config/yaml



Now let's customize the configuration for our specific needs. We'll:
1. Enable L1 adaptive control
2. Add disturbance for testing robustness
3. Adjust simulation time and other parameters


In [3]:
# Customize configuration
config.update({
    # Basic settings
    'simulation_time': 15.0,  # Longer simulation to see effects
    'dt_traj_opt': 50,  # 50ms trajectory optimization timestep
    
    # Enable disturbance
    'enable_disturbance': True,
    'disturbance_start_time': 5.0,  # Start disturbance after 5 seconds
    'disturbance_force': [1.0, 0.5, 0.0],  # Add some external force
    'disturbance_torque': [0.0, 0.0, 0.1],  # Add some external torque
    
    # Enable L1 adaptive control
    'enable_l1_control': True,
    'l1_version': 'v2',  # Use version 2 of L1 controller
    'l1_start_time': 2.0,  # Start L1 control after 2 seconds
    'l1_as_coef': -0.01,  # Adaptation gain
    'l1_filter_time_constant': [0.3, 0.3, 0.3],  # Filter time constants
    
    # Enable noise for realism
    'enable_noise': True,
    'state_noise_std': 0.001,  # Small state measurement noise
    
    # Save and plot settings
    'save_results': True,
    'plot_results': True,
    'results_dir': 'results/numeric_sim'
})

# Save the updated configuration
with open(config_file, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Updated configuration:")
print(yaml.dump(config, default_flow_style=False))


Updated configuration:
disturbance_force:
- 1.0
- 0.5
- 0.0
disturbance_start_time: 5.0
disturbance_torque:
- 0.0
- 0.0
- 0.1
dt_traj_opt: 50
enable_disturbance: true
enable_l1_control: true
enable_noise: true
initial_state:
- 0.0
- 0.0
- 1.0
- 0.0
- 0.0
- 0.0
- 1.0
- -1.2
- -0.6
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
- 0.0
l1_as_coef: -0.01
l1_filter_time_constant:
- 0.3
- 0.3
- 0.3
l1_start_time: 2.0
l1_version: v2
plot_results: true
results_dir: results/numeric_sim
robot_name: s500_uam
save_results: true
simulation_time: 15.0
state_noise_std: 0.001
trajectory_name: hover
use_squash: true
yaml_path: config/yaml



## Run Simulation

Now let's create a simulator instance and run the simulation. The simulator will:
1. Initialize the robot model and controllers
2. Run the simulation with the specified parameters
3. Plot results in real-time
4. Save the results to disk


In [ ]:
# 导入tqdm进度条
from tqdm.notebook import tqdm
import time

# 创建模拟器
print("初始化模拟器...")
simulator = NumericSimulator(config_file)

# 显示模拟参数
print(f"\n模拟参数:")
print(f"- 模拟时间: {simulator.simulation_time} 秒")
print(f"- 控制间隔: {simulator.control_dt*1000} ms")
print(f"- 模拟步长: {simulator.simulation_dt*1000} ms")
print(f"- 总步数: {int(simulator.simulation_time/simulator.simulation_dt)}")
print(f"- L1控制: {'启用' if simulator.enable_l1_control else '禁用'}")
print(f"- 外部干扰: {'启用' if simulator.enable_disturbance else '禁用'}")

# 询问是否继续
response = input("\n模拟器初始化完成。输入 'y' 开始模拟，输入其他任意键取消: ")
if response.lower() != 'y':
    print("模拟已取消")
else:
    print("\n开始模拟...")
    start_time = time.time()
    
    # 运行模拟
    simulator.simulate()
    
    # 计算总用时
    total_time = time.time() - start_time
    print(f"\n模拟完成!")
    print(f"总用时: {total_time:.2f} 秒")
    
    # 询问是否显示图表
    response = input("\n是否显示结果图表? (y/n): ")
    if response.lower() == 'y':
        print("\n生成图表...")
        simulator.plot_result()
    
    # 询问是否保存结果
    response = input("\n是否保存结果? (y/n): ")
    if response.lower() == 'y':
        print("\n保存结果...")
        save_dir = simulator.save_result()
        print(f"结果已保存到: {save_dir}")
    
print("\n模拟会话结束")


Loaded configuration from: numeric_sim_config.yaml
Robot model loaded: s500_uam
  nq (position DOF): 9
  nv (velocity DOF): 8
  State dimension: 17
  Control dimension: 8
[EAGLE_MPC | INFO]: Parsing /home/helei/catkin_eagle_mpc/src/eagle_mpc_debugger/config/yaml/trajectories/temp_trajectory.yaml 
[EAGLE_MPC | INFO]: Parsing /home/helei/catkin_eagle_mpc/src/eagle_mpc_debugger/config/yaml/multicopter/s500.yaml 
[EAGLE_MPC | INFO]: Number of rotors: 4 
[EAGLE_MPC | ERROR]: Could not retrieve dt for weight scaling, using default factor = 1.0. Error: The following key: 'problem_params/dt' has not been found in the parameters server. 
[EAGLE_MPC | INFO]: Creating problem for the given stages. Contact Trajectory = 0 
[EAGLE_MPC | INFO]: Create Problem; navigation# Knots: 20 
[EAGLE_MPC | INFO]: Created terminal model with dt=0 (no time integration) 
iter    cost       merit      stop       grad       preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp  

Deprecated ActuationModelMultiCopterBase: Use ActuationModelFloatingBaseThrusters
Deprecated ActuationModelMultiCopterBase: Use ActuationModelFloatingBaseThrusters


Step   1940, Time  1.940s, MPC solved in   0.45ms, Cost:    5.300, Traj index: 20
Step   1960, Time  1.960s, MPC solved in   0.45ms, Cost:    4.943, Traj index: 20
Step   1980, Time  1.980s, MPC solved in   0.45ms, Cost:    4.399, Traj index: 20
Progress:  13.3%, Elapsed:   0.06s, Avg MPC time:   0.48ms
Step   2000, Time  2.000s, MPC solved in   0.45ms, Cost:    4.199, Traj index: 20
Step   2020, Time  2.020s, MPC:   0.44ms, L1:   0.29ms, Cost:    3.741, Traj: 20
Step   2040, Time  2.040s, MPC:   0.48ms, L1:   0.10ms, Cost:    3.584, Traj: 20
Step   2060, Time  2.060s, MPC:   0.45ms, L1:   0.09ms, Cost:    3.622, Traj: 20
Step   2080, Time  2.080s, MPC:   0.46ms, L1:   0.09ms, Cost:    3.689, Traj: 20
Step   2100, Time  2.100s, MPC:   0.46ms, L1:   0.09ms, Cost:    3.924, Traj: 20
Step   2120, Time  2.120s, MPC:   0.45ms, L1:   0.09ms, Cost:    3.919, Traj: 20
Step   2140, Time  2.140s, MPC:   0.46ms, L1:   0.09ms, Cost:    3.883, Traj: 20
Step   2160, Time  2.160s, MPC:   0.45ms, L1: 

## Analysis Functions

Let's create some helper functions to analyze the simulation results. These functions will help us:
1. Load saved results
2. Create custom plots
3. Calculate performance metrics
4. Compare different simulation runs


In [ ]:
def load_simulation_results(results_dir):
    """Load simulation results from a directory"""
    # Load numpy arrays
    time_data = np.load(os.path.join(results_dir, 'time_data.npy'))
    state_data = np.load(os.path.join(results_dir, 'state_data.npy'))
    control_data = np.load(os.path.join(results_dir, 'control_data.npy'))
    mpc_solve_times = np.load(os.path.join(results_dir, 'mpc_solve_times.npy'))
    mpc_costs = np.load(os.path.join(results_dir, 'mpc_costs.npy'))
    trajectory_index_data = np.load(os.path.join(results_dir, 'trajectory_index_data.npy'))
    
    # Load L1 data if available
    l1_data = {}
    l1_files = ['l1_u_baseline_data.npy', 'l1_u_ad_data.npy', 'l1_compute_times.npy',
                'l1_z_tilde_data.npy', 'l1_sig_hat_data.npy', 'l1_z_hat_data.npy']
    for file in l1_files:
        try:
            l1_data[file.replace('.npy', '')] = np.load(os.path.join(results_dir, file))
        except:
            pass
    
    # Load configuration
    with open(os.path.join(results_dir, 'config.yaml'), 'r') as f:
        config = yaml.safe_load(f)
    
    return {
        'time': time_data,
        'state': state_data,
        'control': control_data,
        'mpc_solve_times': mpc_solve_times,
        'mpc_costs': mpc_costs,
        'trajectory_index': trajectory_index_data,
        'l1_data': l1_data,
        'config': config
    }

def plot_tracking_performance(results):
    """Plot position tracking performance"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Position Tracking Performance', fontsize=16)
    
    # Plot positions
    for i in range(3):
        ax = axes[0, 0]
        ax.plot(results['time'], results['state'][:, i], label=f'Axis {i}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (m)')
    ax.legend()
    ax.grid(True)
    
    # Plot tracking error
    error = np.linalg.norm(results['state'][:, :3], axis=1)
    axes[0, 1].plot(results['time'], error)
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Position Error (m)')
    axes[0, 1].grid(True)
    
    # Plot MPC performance
    axes[1, 0].plot(results['time'][::20], results['mpc_costs'])
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('MPC Cost')
    axes[1, 0].grid(True)
    
    # Plot solve times
    axes[1, 1].plot(results['time'][::20], results['mpc_solve_times'] * 1000)
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Solve Time (ms)')
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

def plot_l1_performance(results):
    """Plot L1 adaptive control performance"""
    if not results['l1_data']:
        print("No L1 data available")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('L1 Adaptive Control Performance', fontsize=16)
    
    # Plot adaptive control inputs
    l1_time = np.arange(len(results['l1_data']['l1_u_ad_data'])) * 0.02  # 20ms control interval
    for i in range(min(3, results['l1_data']['l1_u_ad_data'].shape[1])):
        axes[0, 0].plot(l1_time, results['l1_data']['l1_u_ad_data'][:, i], label=f'Axis {i}')
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Adaptive Control (N)')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Plot baseline vs adaptive control
    axes[0, 1].plot(l1_time, np.linalg.norm(results['l1_data']['l1_u_baseline_data'], axis=1), 
                   label='Baseline')
    axes[0, 1].plot(l1_time, np.linalg.norm(results['l1_data']['l1_u_ad_data'], axis=1), 
                   label='Adaptive')
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Control Magnitude (N)')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Plot state prediction error if available
    if 'l1_z_tilde_data' in results['l1_data']:
        for i in range(min(3, results['l1_data']['l1_z_tilde_data'].shape[1])):
            axes[1, 0].plot(l1_time, results['l1_data']['l1_z_tilde_data'][:, i], label=f'Axis {i}')
        axes[1, 0].set_xlabel('Time (s)')
        axes[1, 0].set_ylabel('State Prediction Error')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    # Plot computation time
    axes[1, 1].plot(l1_time, results['l1_data']['l1_compute_times'] * 1000)
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('L1 Compute Time (ms)')
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

def calculate_performance_metrics(results):
    """Calculate various performance metrics"""
    metrics = {
        'avg_position_error': np.mean(np.linalg.norm(results['state'][:, :3], axis=1)),
        'max_position_error': np.max(np.linalg.norm(results['state'][:, :3], axis=1)),
        'avg_mpc_cost': np.mean(results['mpc_costs']),
        'max_mpc_cost': np.max(results['mpc_costs']),
        'avg_solve_time_ms': np.mean(results['mpc_solve_times']) * 1000,
        'max_solve_time_ms': np.max(results['mpc_solve_times']) * 1000
    }
    
    if results['l1_data']:
        metrics.update({
            'avg_adaptive_control': np.mean(np.linalg.norm(results['l1_data']['l1_u_ad_data'], axis=1)),
            'max_adaptive_control': np.max(np.linalg.norm(results['l1_data']['l1_u_ad_data'], axis=1)),
            'avg_l1_compute_time_ms': np.mean(results['l1_data']['l1_compute_times']) * 1000,
            'max_l1_compute_time_ms': np.max(results['l1_data']['l1_compute_times']) * 1000
        })
    
    return metrics
